# Optimizer
Gets data and uses it to optimize a method (imported from method.py) using a loss function and tests it against unseen data.
TODO:
- Cross Validation implementation


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.optim as optim

# Import data

In [2]:
### Load the data
data_path = 'data/survey_2022-06-08.csv'
df = pd.read_csv(data_path)
df

,1. Geschlecht:,2. Alter:,3. Körpergröße (in cm),4. Körpergewicht (in kg),8. Wie lange fahren Sie schon auf dem Bus?,10. Bitte geben Sie an ob Sie sitzen oder stehen:,11. Bitte geben Sie Ihre Position im Bus an:,"12. Bitte geben Sie an, ob sie näher einer Fenster oder dem Gang sind:",clothing_insulation,comfort
0,0,62,162,60,0,0,0.0,0,31,1.000000
1,0,36,163,65,0,0,1.0,0,21,1.666667
2,0,23,168,68,1,0,2.0,0,35,1.000000
3,0,22,178,80,0,0,3.0,1,26,1.333333
4,0,25,163,65,0,0,4.0,1,26,1.333333
5,0,27,168,57,2,0,4.0,1,26,1.666667
6,1,38,187,69,0,0,5.0,0,22,1.666667
7,1,73,167,83,0,0,5.0,1,32,2.000000
8,1,61,176,80,0,0,4.0,0,29,1.333333
9,1,47,184,81,1,1,5.0,0,22,1.333333


# Scale Data

In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(df)
y = X[:,-1]
x = X[:,:-1]

# Cross Validation Split #TODO

# Train Test Split

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
import time
from torch.optim.lr_scheduler import ExponentialLR
from methods import *
from torch import nn

epochs = 1000
method = Linear(input_size=x_train.shape[1])
criterion = nn.MSELoss()
params_to_be_optimized = method.get_parameters()
optimizer = optim.Adam(params_to_be_optimized, lr=0.01)
scheduler = ExponentialLR(optimizer, gamma=0.999)

for i in range(epochs):
  iteration_time = time.time()
  optimizer.zero_grad()

  # forward pass
  y_predicted = method.predict(x_train)
  criterion = nn.MSELoss()
  loss = criterion(output, target_data)

  # backward pass
  loss.backward()
  optimizer.step()
  scheduler.step()


  with torch.no_grad():
